In [1]:
from keras.models import load_model
from attn_layer import *
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

model = load_model('bilstm_attention_cnn_V2.h5', custom_objects={'AttentionWithContext' : AttentionWithContext},
                   compile=False)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.










In [7]:
import pandas as pd
import pickle

df = pd.read_pickle('todos_prueba_1.pkl')

In [8]:
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
import string
import gensim

fname = "w2v_dish_1M.kv"

word_model = gensim.models.Word2Vec.load(fname, mmap='r')
pretrained_weights = word_model.wv.vectors


def word2idx(word):
    try:    
        return word_model.wv.vocab[word].index
    except:
        return 0
    
def idx2word(idx):
    return word_model.wv.index2word[idx]


In [12]:
import numpy as np

print('\nPreparing input data...')

sentences = df.tokens
max_length = 30

X = np.zeros([len(sentences), max_length], dtype=np.int32) 
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence[:max_length]):
        X[i, t] = word2idx(word)
            
print('X shape:', X.shape)



Preparing input data...
X shape: (915619, 30)


In [13]:
y_pred = model.predict(X)
y_pred_decoded = np.argmax(y_pred,axis=1)

In [29]:
labels = ['Alitas', 'Arabe', 'Arroces', 'Asiática otros', 'Barbacoa',
       'Bebidas', 'Boneless', 'Botanas otros', 'Bowls', 'Burritos',
       'Caldos', 'Cemitas', 'Chilaquiles', 'Comida china',
       'Comida corrida', 'Cortes', 'Costillas', 'Crepas',
       'Desayunos otros', 'Empanadas', 'Enchiladas', 'Ensalada',
       'Flautas', 'Fruta', 'Garnachas otros', 'Gorditas', 'Hamburguesa',
       'Helados', 'Hot cakes y waffles', 'Hot dog', 'Huevos y omelet',
       'Internacional otros', 'Italiana', 'Mexicana otros', 'Milanesa',
       'Nuggets', 'Otros otros', 'Panes', 'Panini', 'Papas', 'Pasta',
       'Pasteles', 'Pastor', 'Pescado y mariscos', 'Pizza', 'Pollo',
       'Postres otros', 'Pozole', 'Quesadillas', 'Sandwich', 'Sopas',
       'Sushi', 'Tacos otros', 'Tamales', 'Torta', 'Tostadas', 'Vegano']

In [15]:
y_pred[0]

array([2.8887758e-04, 5.7059620e-04, 4.7420730e-05, 2.2994697e-03,
       3.2162928e-04, 1.1931306e-04, 8.1523045e-05, 1.7575240e-02,
       2.2032161e-04, 5.7285646e-04, 5.6217996e-05, 4.5952282e-05,
       3.3941740e-04, 1.1897390e-04, 2.7955817e-03, 5.7944166e-04,
       1.7043378e-04, 1.3779203e-05, 2.5421318e-02, 1.0828720e-04,
       2.4727345e-04, 2.9544224e-04, 2.0742283e-04, 2.8035618e-04,
       1.0816299e-03, 1.6597839e-04, 1.5254401e-03, 1.0691789e-04,
       1.6761424e-04, 5.1385915e-04, 1.9916059e-03, 5.8742714e-01,
       1.5269413e-04, 9.6484402e-04, 2.2475612e-03, 7.0825679e-04,
       2.8049138e-03, 6.4773209e-02, 4.7648372e-03, 1.9303465e-02,
       3.1924871e-04, 9.8153418e-05, 6.1770750e-04, 3.1226666e-03,
       4.7253636e-03, 4.8648446e-05, 7.4412458e-04, 4.6864057e-05,
       6.0443900e-04, 3.4525995e-03, 2.3200024e-04, 2.1981383e-05,
       8.7480405e-03, 1.2241816e-04, 2.3476467e-01, 2.6150930e-04,
       5.9248420e-04], dtype=float32)

In [20]:
prob = [y_pred[i][y_pred_decoded[i]] for i in range(len(y_pred))]

In [22]:
df['predicted'] = y_pred_decoded
df['probability'] = prob

In [30]:
str_label = [labels[prd] for prd in y_pred_decoded]

In [31]:
df['predicted_label'] = str_label

In [32]:
df[['frn_id','item_code','category','item_name','item_description','predicted','probability','predicted_label']].to_csv('tax_etiquetados_prueba0130_ALL.csv')